In [1]:
import os
import tempfile
import numpy as np
import pandas as pd
import cv2
import mediapipe as mp
import matplotlib.pyplot as plt
import pickle
import math
import catboost as cb

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

In [2]:
train_dir = 'data/train'
# 비디오 파일 목록과 태그를 포함하는 리스트를 만드는 함수
def create_data_list(data_dir):
    data_list = []
    for item in os.listdir(data_dir):
        item_path = os.path.join(data_dir, item)
        if os.path.isdir(item_path):
            for file_name in os.listdir(item_path):
                if file_name.endswith('.jpg'):
                    data_list.append((item, str(data_dir+'/'+item)+'/'+file_name))
    return data_list

train_list = create_data_list(train_dir)
train_df = pd.DataFrame(data=train_list, columns=['tag', 'image_name'])
train_df.to_csv('train_data.csv', encoding='utf-8-sig', index=False)
print(f"학습 데이터: {len(train_df)}")

학습 데이터: 0


In [2]:
# 손가락 각도 계산 함수
def calculate_angles(hand_landmarks, image_shape):
    joint = np.zeros((21, 3))
    for j, lm in enumerate(hand_landmarks.landmark):
        joint[j] = [lm.x * image_shape[1], lm.y * image_shape[0], lm.z]
        
    # 벡터 계산
    v1 = joint[[0,1,2,3,0,5,6,7,0,9,10,11,0,13,14,15,0,17,18,19],:]
    v2 = joint[[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20],:]
    v = v2 - v1
    v = v / np.linalg.norm(v, axis=1)[:, np.newaxis]

    # 각도 계산
    angle = np.arccos(np.einsum('nt,nt->n', v[[0,1,2,4,5,6,8,9,10,12,13,14,16,17,18],:], v[[1,2,3,5,6,7,9,10,11,13,14,15,17,18,19],:]))
    angle = np.degrees(angle)
    
    # NaN 값이 있을 경우 0으로 대체
    angle = np.nan_to_num(angle)
    
    return angle, joint.flatten()

In [3]:
def calculate_pose_angles(pose_landmarks, image_shape):
    joint = np.zeros((33, 3))
    for j, lm in enumerate(pose_landmarks.landmark):
        joint[j] = [lm.x * image_shape[1], lm.y * image_shape[0], lm.z]

    # 팔 각도 계산: 어깨(11, 12), 팔꿈치(13, 14), 손목(15, 16) 랜드마크 사용
    v1 = joint[[11, 13, 12, 14], :]  # 어깨와 팔꿈치
    v2 = joint[[13, 15, 14, 16], :]  # 팔꿈치와 손목
    v = v2 - v1
    v = v / np.linalg.norm(v, axis=1)[:, np.newaxis]

    # 양팔의 각도 계산
    arm_angles = np.arccos(np.einsum('nt,nt->n', v[[0, 2], :], v[[1, 3], :]))
    arm_angles = np.degrees(arm_angles)

    return arm_angles

In [5]:
df_data = pd.DataFrame()

In [4]:
def create_training_data_from_image(image_path, label):
    global df_data
    mp_hands = mp.solutions.hands
    hands = mp_hands.Hands(static_image_mode=True, max_num_hands=2, min_detection_confidence=0.5)
    mp_pose = mp.solutions.pose
    pose = mp_pose.Pose(static_image_mode=True, min_detection_confidence=0.5)
    
    # 이미지 파일 불러오기
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    result = hands.process(img)
    pose_result = pose.process(img)

    landmark_data = []
    arm_data_row = [0] * 2
    right_hand_angles_row = [0] * 15  # 오른손 각도
    left_hand_angles_row = [0] * 15   # 왼손 각도
    right_hand_coords_row = [0] * 63  # 오른손 좌표
    left_hand_coords_row = [0] * 63   # 왼손 좌표
    
    if result.multi_hand_landmarks:
        for i, hand_landmarks in enumerate(result.multi_hand_landmarks):
            hand_type = result.multi_handedness[i].classification[0].label
            angles, joint_coords = calculate_angles(hand_landmarks, img.shape[:2])            
            if hand_type == 'Right':
                right_hand_angles_row = angles.tolist()
                right_hand_coords_row = joint_coords.tolist()
            elif hand_type == 'Left':
                left_hand_angles_row = angles.tolist()
                left_hand_coords_row = joint_coords.tolist()
            
    if pose_result.pose_landmarks:
        arm_angles = calculate_pose_angles(pose_result.pose_landmarks, img.shape[:2])
        arm_data_row = arm_angles.tolist()
        for idx in [11, 13, 15, 12, 14, 16]:
            landmark = pose_result.pose_landmarks.landmark[idx]
            landmark_data.extend([landmark.x, landmark.y, landmark.z])

    hand_data_row = right_hand_angles_row + right_hand_coords_row + left_hand_angles_row + left_hand_coords_row
    df_data = pd.concat([df_data, pd.DataFrame([hand_data_row + arm_data_row + landmark_data + [label]])], ignore_index=True)

    return df_data

In [7]:
# 특정 폴더의 모든 이미지를 처리하고 CSV 파일로 저장하는 함수
def process_folder(folder_path, label):
    df_folder = pd.DataFrame()

    # 폴더 내 모든 이미지 처리
    for image_name in os.listdir(folder_path):
        image_path = os.path.join(folder_path, image_name)
        df_folder = create_training_data_from_image(image_path, label)

    # 컬럼 이름 설정
    right_hand_angle_columns = [f'right_hand_angle_{i}' for i in range(1, 16)]  # 오른손 각도
    right_hand_coord_columns = [f'right_hand_coord_{i}_{axis}' for i in range(1, 22) for axis in ['x', 'y', 'z']]  # 오른손 좌표
    left_hand_angle_columns = [f'left_hand_angle_{i}' for i in range(1, 16)]  # 왼손 각도
    left_hand_coord_columns = [f'left_hand_coord_{i}_{axis}' for i in range(1, 22) for axis in ['x', 'y', 'z']]  # 왼손 좌표
    arm_angle_columns = ['arm_1', 'arm_2']  # 팔 각도
    other_landmark_columns = [f'pose_{idx}_{axis}' for idx in [11, 13, 15, 12, 14, 16] for axis in ['x', 'y', 'z']]  # 포즈 랜드마크

    all_columns = right_hand_angle_columns + right_hand_coord_columns + left_hand_angle_columns + left_hand_coord_columns + arm_angle_columns + other_landmark_columns + ['label']
    df_folder.columns = all_columns
    df_folder.to_csv(f'{label}_data.csv', index=False)

In [8]:
# 'data/train' 폴더 내 모든 하위 폴더를 순회
base_folder = 'data/train'
for label_folder in os.listdir(base_folder):
    folder_path = base_folder+"/"+label_folder
    if os.path.isdir(folder_path):
        print(folder_path)
        process_folder(folder_path, label_folder)

data/train/where


In [4]:
df_data = pd.read_csv("final_training_data.csv")

In [5]:
df_data

,right_hand_angle_1,right_hand_angle_2,right_hand_angle_3,right_hand_angle_4,right_hand_angle_5,right_hand_angle_6,right_hand_angle_7,right_hand_angle_8,right_hand_angle_9,right_hand_angle_10,...,pose_12_x,pose_12_y,pose_12_z,pose_14_x,pose_14_y,pose_14_z,pose_16_x,pose_16_y,pose_16_z,label
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.304181,0.269859,-0.348680,0.166155,0.444097,-0.433554,0.386356,0.487853,-0.751229,belly
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.308368,0.272443,-0.268009,0.169883,0.445997,-0.414924,0.379054,0.481162,-0.839744,belly
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.311658,0.272051,-0.334071,0.170422,0.444774,-0.460341,0.380478,0.479141,-0.846146,belly
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.313875,0.270783,-0.190262,0.164895,0.441304,-0.319333,0.378377,0.476458,-0.652272,belly
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.316910,0.266524,-0.211217,0.170624,0.440041,-0.354612,0.381129,0.475872,-0.706072,belly
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1662,29.153906,37.521067,0.193556,2.650470,1.648691,1.943038,18.448943,3.891265,0.887181,23.433697,...,0.290881,0.321418,-0.291566,0.174090,0.502585,-0.371022,0.348492,0.540278,-0.797737,wallet
1663,39.600530,33.992821,4.371888,1.726375,0.816531,5.683913,15.128424,5.037508,4.550825,19.762297,...,0.454027,0.349702,-0.255577,0.357657,0.528405,-0.318673,0.490959,0.558559,-0.771193,wallet
1664,38.216025,34.279362,10.160899,7.723580,3.223601,4.792514,17.599725,5.970413,5.148321,18.454904,...,0.356184,0.388115,-0.222924,0.280295,0.541145,-0.254589,0.388877,0.567603,-0.621091,wallet
1665,38.909594,35.486527,3.066746,4.038148,2.088028,6.664855,18.082172,7.164371,5.325536,20.892320,...,0.446762,0.335734,-0.289392,0.353480,0.527142,-0.319838,0.493551,0.552985,-0.720406,wallet


In [6]:
train_df, test_df = train_test_split(df_data, test_size=0.2, random_state=42)

In [7]:
train_df

,right_hand_angle_1,right_hand_angle_2,right_hand_angle_3,right_hand_angle_4,right_hand_angle_5,right_hand_angle_6,right_hand_angle_7,right_hand_angle_8,right_hand_angle_9,right_hand_angle_10,...,pose_12_x,pose_12_y,pose_12_z,pose_14_x,pose_14_y,pose_14_z,pose_16_x,pose_16_y,pose_16_z,label
1107,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.495527,0.484649,-0.101714,0.370429,0.600202,-0.365068,0.392948,0.477436,-0.892347,report
266,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.395336,0.353485,-0.194129,0.367314,0.503135,-0.390118,0.387543,0.497928,-0.919688,where
148,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.260899,0.295553,-0.183296,0.120585,0.460280,-0.343750,0.315324,0.524766,-0.690026,belly
433,20.373921,16.201404,6.309923,5.015151,15.876007,16.235372,13.165958,29.446065,17.401332,11.390159,...,0.360765,0.557879,-0.123458,0.111813,0.685621,-0.539459,0.249732,0.507656,-1.003286,child
874,29.614590,14.299186,8.243904,171.926897,24.038474,6.364953,160.425506,29.844250,8.873070,143.093250,...,0.374893,0.398790,-0.166987,0.309472,0.560965,-0.474105,0.507887,0.562636,-0.994990,plz
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1130,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.584476,0.470150,-0.101448,0.455432,0.588735,-0.354781,0.485134,0.455438,-0.825966,report
1294,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.508064,0.369799,-0.149947,0.480926,0.515244,-0.009838,0.508652,0.565454,-0.220193,sick
860,30.701336,13.772301,10.418864,170.743456,24.544216,9.835080,159.284978,29.628616,15.303178,142.333430,...,0.265513,0.372473,-0.137195,0.195048,0.534810,-0.404216,0.400319,0.531868,-0.886087,plz
1459,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.450431,0.448579,-0.036746,0.366772,0.627043,-0.193774,0.395305,0.518379,-0.628783,toilet


In [8]:
X_train = train_df.drop('label', axis =1)
y_train = train_df['label']

In [9]:
X_test = test_df.drop('label', axis =1)
y_test = test_df['label']

In [10]:
lgbm_model = LGBMClassifier(random_state=200, n_jobs=-1, reg_alpha=0.9, reg_lambda=0.2, n_estimators=200)
lgbm_model.fit(X_train, y_train)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.004608 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 38327
[LightGBM] [Info] Number of data points in the train set: 1333, number of used features: 176
[LightGBM] [Info] Start training from score -2.275206
[LightGBM] [Info] Start training from score -2.358905
[LightGBM] [Info] Start training from score -2.335375
[LightGBM] [Info] Start training from score -2.327653
[LightGBM] [Info] Start training from score -2.319990
[LightGBM] [Info] Start training from score -2.289913
[LightGBM] [Info] Start training from score -2.358905
[LightGBM] [Info] Start training from score -2.218454
[LightGBM] [Info] Start training from score -2.253545
[LightGBM] [Info] Start training from score -2.297348
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning]

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

LGBMClassifier(n_estimators=200, n_jobs=-1, random_state=200, reg_alpha=0.9,
               reg_lambda=0.2)

In [11]:
result = cross_val_score(lgbm_model,
                          X_test,
                          y_test,
                          cv = 5)
result.mean()

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001341 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 11482
[LightGBM] [Info] Number of data points in the train set: 267, number of used features: 176
[LightGBM] [Info] Start training from score -2.368373
[LightGBM] [Info] Start training from score -2.060888
[LightGBM] [Info] Start training from score -2.186051
[LightGBM] [Info] Start training from score -2.219953
[LightGBM] [Info] Start training from score -2.409195
[LightGBM] [Info] Start training from score -2.329152
[LightGBM] [Info] Start training from score -2.153261
[LightGBM] [Info] Start training from score -2.591516
[LightGBM] [Info] Start training from score -2.591516
[LightGBM] [Info] Start training from score -2.255044
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] 

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001135 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 11490
[LightGBM] [Info] Number of data points in the train set: 267, number of used features: 176
[LightGBM] [Info] Start training from score -2.329152
[LightGBM] [Info] Start training from score -2.090741
[LightGBM] [Info] Start training from score -2.186051
[LightGBM] [Info] Start training from score -2.219953
[LightGBM] [Info] Start training from score -2.368373
[LightGBM] [Info] Start training from score -2.329152
[LightGBM] [Info] Start training from score -2.186051
[LightGBM] [Info] Start training from score -2.591516
[LightGBM] [Info] Start training from score -2.591516
[LightGBM] [Info] Start training from score -2.255044
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] 

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001205 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 11633
[LightGBM] [Info] Number of data points in the train set: 267, number of used features: 176
[LightGBM] [Info] Start training from score -2.329152
[LightGBM] [Info] Start training from score -2.090741
[LightGBM] [Info] Start training from score -2.153261
[LightGBM] [Info] Start training from score -2.219953
[LightGBM] [Info] Start training from score -2.368373
[LightGBM] [Info] Start training from score -2.368373
[LightGBM] [Info] Start training from score -2.186051
[LightGBM] [Info] Start training from score -2.591516
[LightGBM] [Info] Start training from score -2.591516
[LightGBM] [Info] Start training from score -2.255044
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] 

...

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001280 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 11566
[LightGBM] [Info] Number of data points in the train set: 267, number of used features: 176
[LightGBM] [Info] Start training from score -2.329152
[LightGBM] [Info] Start training from score -2.060888
[LightGBM] [Info] Start training from score -2.153261
[LightGBM] [Info] Start training from score -2.255044
[LightGBM] [Info] Start training from score -2.368373
[LightGBM] [Info] Start training from score -2.368373
[LightGBM] [Info] Start training from score -2.186051
[LightGBM] [Info] Start training from score -2.591516
[LightGBM] [Info] Start training from score -2.591516
[LightGBM] [Info] Start training from score -2.255044
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] 

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001502 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 11617
[LightGBM] [Info] Number of data points in the train set: 268, number of used features: 176
[LightGBM] [Info] Start training from score -2.372111
[LightGBM] [Info] Start training from score -2.064626
[LightGBM] [Info] Start training from score -2.189790
[LightGBM] [Info] Start training from score -2.223691
[LightGBM] [Info] Start training from score -2.372111
[LightGBM] [Info] Start training from score -2.332890
[LightGBM] [Info] Start training from score -2.157000
[LightGBM] [Info] Start training from score -2.595255
[LightGBM] [Info] Start training from score -2.595255
[LightGBM] [Info] Start training from score -2.258782
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] 

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

0.982089552238806

In [12]:
lgbm_model.booster_.save_model('lgbm_model.txt')

In [13]:
cb_model = CatBoostClassifier()
cb_model.fit(X_train, y_train)

Learning rate set to 0.080295
0:	learn: 1.8465147	total: 261ms	remaining: 4m 20s
1:	learn: 1.6118919	total: 372ms	remaining: 3m 5s
2:	learn: 1.3903992	total: 486ms	remaining: 2m 41s
3:	learn: 1.2325709	total: 600ms	remaining: 2m 29s
4:	learn: 1.1029226	total: 713ms	remaining: 2m 21s
5:	learn: 0.9869564	total: 847ms	remaining: 2m 20s
6:	learn: 0.8907935	total: 1.01s	remaining: 2m 23s
7:	learn: 0.8092498	total: 1.16s	remaining: 2m 23s
8:	learn: 0.7400986	total: 1.27s	remaining: 2m 20s
9:	learn: 0.6804977	total: 1.38s	remaining: 2m 16s
10:	learn: 0.6260258	total: 1.49s	remaining: 2m 14s
11:	learn: 0.5763274	total: 1.6s	remaining: 2m 11s
12:	learn: 0.5315807	total: 1.71s	remaining: 2m 10s
13:	learn: 0.4932307	total: 1.86s	remaining: 2m 10s
14:	learn: 0.4590342	total: 2.03s	remaining: 2m 13s
15:	learn: 0.4255021	total: 2.15s	remaining: 2m 12s
16:	learn: 0.3959705	total: 2.27s	remaining: 2m 11s
17:	learn: 0.3697579	total: 2.38s	remaining: 2m 9s
18:	learn: 0.3454553	total: 2.49s	remaining: 2m

160:	learn: 0.0131732	total: 21.8s	remaining: 1m 53s
161:	learn: 0.0131403	total: 22s	remaining: 1m 53s
162:	learn: 0.0130961	total: 22.1s	remaining: 1m 53s
163:	learn: 0.0129452	total: 22.2s	remaining: 1m 53s
164:	learn: 0.0127725	total: 22.3s	remaining: 1m 53s
165:	learn: 0.0125422	total: 22.5s	remaining: 1m 52s
166:	learn: 0.0124065	total: 22.6s	remaining: 1m 52s
167:	learn: 0.0123265	total: 22.7s	remaining: 1m 52s
168:	learn: 0.0121845	total: 22.9s	remaining: 1m 52s
169:	learn: 0.0121457	total: 23.1s	remaining: 1m 52s
170:	learn: 0.0120274	total: 23.2s	remaining: 1m 52s
171:	learn: 0.0120094	total: 23.3s	remaining: 1m 52s
172:	learn: 0.0118911	total: 23.5s	remaining: 1m 52s
173:	learn: 0.0118037	total: 23.6s	remaining: 1m 51s
174:	learn: 0.0116792	total: 23.7s	remaining: 1m 51s
175:	learn: 0.0115936	total: 23.9s	remaining: 1m 51s
176:	learn: 0.0115612	total: 24s	remaining: 1m 51s
177:	learn: 0.0114739	total: 24.2s	remaining: 1m 51s
178:	learn: 0.0113304	total: 24.3s	remaining: 1m 5

316:	learn: 0.0057531	total: 43.2s	remaining: 1m 33s
317:	learn: 0.0057269	total: 43.3s	remaining: 1m 32s
318:	learn: 0.0056734	total: 43.5s	remaining: 1m 32s
319:	learn: 0.0056670	total: 43.6s	remaining: 1m 32s
320:	learn: 0.0056620	total: 43.7s	remaining: 1m 32s
321:	learn: 0.0056292	total: 43.9s	remaining: 1m 32s
322:	learn: 0.0055966	total: 44.1s	remaining: 1m 32s
323:	learn: 0.0055868	total: 44.2s	remaining: 1m 32s
324:	learn: 0.0055719	total: 44.3s	remaining: 1m 32s
325:	learn: 0.0055644	total: 44.4s	remaining: 1m 31s
326:	learn: 0.0055235	total: 44.6s	remaining: 1m 31s
327:	learn: 0.0055173	total: 44.7s	remaining: 1m 31s
328:	learn: 0.0054782	total: 44.9s	remaining: 1m 31s
329:	learn: 0.0054706	total: 45s	remaining: 1m 31s
330:	learn: 0.0054666	total: 45.1s	remaining: 1m 31s
331:	learn: 0.0054454	total: 45.3s	remaining: 1m 31s
332:	learn: 0.0054398	total: 45.4s	remaining: 1m 30s
333:	learn: 0.0054299	total: 45.5s	remaining: 1m 30s
334:	learn: 0.0053890	total: 45.7s	remaining: 1m

472:	learn: 0.0034747	total: 1m 4s	remaining: 1m 12s
473:	learn: 0.0034583	total: 1m 4s	remaining: 1m 11s
474:	learn: 0.0034569	total: 1m 4s	remaining: 1m 11s
475:	learn: 0.0034502	total: 1m 5s	remaining: 1m 11s
476:	learn: 0.0034350	total: 1m 5s	remaining: 1m 11s
477:	learn: 0.0034336	total: 1m 5s	remaining: 1m 11s
478:	learn: 0.0034236	total: 1m 5s	remaining: 1m 11s
479:	learn: 0.0034072	total: 1m 5s	remaining: 1m 11s
480:	learn: 0.0033929	total: 1m 5s	remaining: 1m 10s
481:	learn: 0.0033867	total: 1m 5s	remaining: 1m 10s
482:	learn: 0.0033855	total: 1m 6s	remaining: 1m 10s
483:	learn: 0.0033730	total: 1m 6s	remaining: 1m 10s
484:	learn: 0.0033699	total: 1m 6s	remaining: 1m 10s
485:	learn: 0.0033559	total: 1m 6s	remaining: 1m 10s
486:	learn: 0.0033523	total: 1m 6s	remaining: 1m 10s
487:	learn: 0.0033414	total: 1m 6s	remaining: 1m 10s
488:	learn: 0.0033227	total: 1m 6s	remaining: 1m 9s
489:	learn: 0.0033217	total: 1m 6s	remaining: 1m 9s
490:	learn: 0.0033197	total: 1m 7s	remaining: 1m

628:	learn: 0.0024405	total: 1m 25s	remaining: 50.7s
629:	learn: 0.0024382	total: 1m 26s	remaining: 50.6s
630:	learn: 0.0024310	total: 1m 26s	remaining: 50.4s
631:	learn: 0.0024302	total: 1m 26s	remaining: 50.3s
632:	learn: 0.0024234	total: 1m 26s	remaining: 50.2s
633:	learn: 0.0024186	total: 1m 26s	remaining: 50s
634:	learn: 0.0024158	total: 1m 26s	remaining: 49.9s
635:	learn: 0.0024124	total: 1m 26s	remaining: 49.8s
636:	learn: 0.0024068	total: 1m 27s	remaining: 49.6s
637:	learn: 0.0024059	total: 1m 27s	remaining: 49.5s
638:	learn: 0.0024053	total: 1m 27s	remaining: 49.3s
639:	learn: 0.0023997	total: 1m 27s	remaining: 49.2s
640:	learn: 0.0023973	total: 1m 27s	remaining: 49.1s
641:	learn: 0.0023949	total: 1m 27s	remaining: 48.9s
642:	learn: 0.0023851	total: 1m 27s	remaining: 48.8s
643:	learn: 0.0023785	total: 1m 28s	remaining: 48.7s
644:	learn: 0.0023754	total: 1m 28s	remaining: 48.5s
645:	learn: 0.0023685	total: 1m 28s	remaining: 48.4s
646:	learn: 0.0023676	total: 1m 28s	remaining: 4

785:	learn: 0.0018717	total: 1m 47s	remaining: 29.2s
786:	learn: 0.0018683	total: 1m 47s	remaining: 29.1s
787:	learn: 0.0018679	total: 1m 47s	remaining: 28.9s
788:	learn: 0.0018619	total: 1m 47s	remaining: 28.8s
789:	learn: 0.0018613	total: 1m 47s	remaining: 28.7s
790:	learn: 0.0018598	total: 1m 47s	remaining: 28.5s
791:	learn: 0.0018541	total: 1m 48s	remaining: 28.4s
792:	learn: 0.0018504	total: 1m 48s	remaining: 28.2s
793:	learn: 0.0018466	total: 1m 48s	remaining: 28.1s
794:	learn: 0.0018417	total: 1m 48s	remaining: 28s
795:	learn: 0.0018359	total: 1m 48s	remaining: 27.8s
796:	learn: 0.0018307	total: 1m 48s	remaining: 27.7s
797:	learn: 0.0018277	total: 1m 48s	remaining: 27.6s
798:	learn: 0.0018274	total: 1m 49s	remaining: 27.4s
799:	learn: 0.0018263	total: 1m 49s	remaining: 27.3s
800:	learn: 0.0018255	total: 1m 49s	remaining: 27.1s
801:	learn: 0.0018214	total: 1m 49s	remaining: 27s
802:	learn: 0.0018172	total: 1m 49s	remaining: 26.9s
803:	learn: 0.0018116	total: 1m 49s	remaining: 26.

943:	learn: 0.0014850	total: 2m 8s	remaining: 7.64s
944:	learn: 0.0014847	total: 2m 8s	remaining: 7.5s
945:	learn: 0.0014840	total: 2m 9s	remaining: 7.37s
946:	learn: 0.0014835	total: 2m 9s	remaining: 7.23s
947:	learn: 0.0014819	total: 2m 9s	remaining: 7.09s
948:	learn: 0.0014803	total: 2m 9s	remaining: 6.96s
949:	learn: 0.0014767	total: 2m 9s	remaining: 6.82s
950:	learn: 0.0014753	total: 2m 9s	remaining: 6.69s
951:	learn: 0.0014747	total: 2m 9s	remaining: 6.55s
952:	learn: 0.0014733	total: 2m 10s	remaining: 6.41s
953:	learn: 0.0014696	total: 2m 10s	remaining: 6.28s
954:	learn: 0.0014686	total: 2m 10s	remaining: 6.14s
955:	learn: 0.0014652	total: 2m 10s	remaining: 6s
956:	learn: 0.0014630	total: 2m 10s	remaining: 5.87s
957:	learn: 0.0014623	total: 2m 10s	remaining: 5.73s
958:	learn: 0.0014590	total: 2m 10s	remaining: 5.59s
959:	learn: 0.0014587	total: 2m 10s	remaining: 5.46s
960:	learn: 0.0014568	total: 2m 11s	remaining: 5.32s
961:	learn: 0.0014562	total: 2m 11s	remaining: 5.18s
962:	l

In [14]:
result = cross_val_score(cb_model,
                          X_test,
                          y_test,
                          cv = 5)
result.mean()

Learning rate set to 0.073973
0:	learn: 2.1340082	total: 81.5ms	remaining: 1m 21s
1:	learn: 1.9430127	total: 165ms	remaining: 1m 22s
2:	learn: 1.7938698	total: 248ms	remaining: 1m 22s
3:	learn: 1.6574315	total: 331ms	remaining: 1m 22s
4:	learn: 1.5443425	total: 411ms	remaining: 1m 21s
5:	learn: 1.4298400	total: 493ms	remaining: 1m 21s
6:	learn: 1.3353483	total: 572ms	remaining: 1m 21s
7:	learn: 1.2472180	total: 654ms	remaining: 1m 21s
8:	learn: 1.1709000	total: 735ms	remaining: 1m 20s
9:	learn: 1.1114320	total: 842ms	remaining: 1m 23s
10:	learn: 1.0501472	total: 957ms	remaining: 1m 26s
11:	learn: 0.9872559	total: 1.08s	remaining: 1m 28s
12:	learn: 0.9370279	total: 1.17s	remaining: 1m 28s
13:	learn: 0.8922764	total: 1.25s	remaining: 1m 27s
14:	learn: 0.8437594	total: 1.33s	remaining: 1m 27s
15:	learn: 0.7994371	total: 1.41s	remaining: 1m 26s
16:	learn: 0.7583206	total: 1.49s	remaining: 1m 26s
17:	learn: 0.7199074	total: 1.57s	remaining: 1m 25s
18:	learn: 0.6854463	total: 1.65s	remaining

158:	learn: 0.0339859	total: 14.2s	remaining: 1m 15s
159:	learn: 0.0336235	total: 14.3s	remaining: 1m 15s
160:	learn: 0.0334278	total: 14.4s	remaining: 1m 15s
161:	learn: 0.0331179	total: 14.5s	remaining: 1m 14s
162:	learn: 0.0328043	total: 14.6s	remaining: 1m 14s
163:	learn: 0.0324749	total: 14.7s	remaining: 1m 14s
164:	learn: 0.0322348	total: 14.7s	remaining: 1m 14s
165:	learn: 0.0319204	total: 14.9s	remaining: 1m 14s
166:	learn: 0.0315635	total: 15s	remaining: 1m 14s
167:	learn: 0.0312465	total: 15.1s	remaining: 1m 14s
168:	learn: 0.0309270	total: 15.2s	remaining: 1m 14s
169:	learn: 0.0306715	total: 15.2s	remaining: 1m 14s
170:	learn: 0.0303638	total: 15.3s	remaining: 1m 14s
171:	learn: 0.0301068	total: 15.4s	remaining: 1m 14s
172:	learn: 0.0298226	total: 15.5s	remaining: 1m 13s
173:	learn: 0.0295251	total: 15.6s	remaining: 1m 13s
174:	learn: 0.0292681	total: 15.6s	remaining: 1m 13s
175:	learn: 0.0290854	total: 15.7s	remaining: 1m 13s
176:	learn: 0.0288332	total: 15.9s	remaining: 1m

318:	learn: 0.0131652	total: 28.4s	remaining: 1m
319:	learn: 0.0131231	total: 28.5s	remaining: 1m
320:	learn: 0.0130757	total: 28.6s	remaining: 1m
321:	learn: 0.0130223	total: 28.7s	remaining: 1m
322:	learn: 0.0129647	total: 28.8s	remaining: 1m
323:	learn: 0.0129326	total: 28.9s	remaining: 1m
324:	learn: 0.0128782	total: 29s	remaining: 1m
325:	learn: 0.0128291	total: 29.1s	remaining: 1m
326:	learn: 0.0127685	total: 29.2s	remaining: 1m
327:	learn: 0.0127122	total: 29.3s	remaining: 60s
328:	learn: 0.0126572	total: 29.4s	remaining: 59.9s
329:	learn: 0.0125968	total: 29.4s	remaining: 59.8s
330:	learn: 0.0125480	total: 29.5s	remaining: 59.7s
331:	learn: 0.0124965	total: 29.6s	remaining: 59.5s
332:	learn: 0.0124491	total: 29.7s	remaining: 59.5s
333:	learn: 0.0123905	total: 29.8s	remaining: 59.4s
334:	learn: 0.0123402	total: 29.9s	remaining: 59.4s
335:	learn: 0.0122930	total: 30s	remaining: 59.3s
336:	learn: 0.0122654	total: 30.1s	remaining: 59.2s
337:	learn: 0.0122087	total: 30.2s	remaining:

478:	learn: 0.0082592	total: 42.8s	remaining: 46.5s
479:	learn: 0.0082409	total: 42.9s	remaining: 46.5s
480:	learn: 0.0082177	total: 43s	remaining: 46.4s
481:	learn: 0.0081915	total: 43.1s	remaining: 46.3s
482:	learn: 0.0081688	total: 43.1s	remaining: 46.2s
483:	learn: 0.0081435	total: 43.2s	remaining: 46.1s
484:	learn: 0.0081252	total: 43.3s	remaining: 46s
485:	learn: 0.0081023	total: 43.4s	remaining: 45.9s
486:	learn: 0.0080754	total: 43.5s	remaining: 45.8s
487:	learn: 0.0080545	total: 43.6s	remaining: 45.7s
488:	learn: 0.0080319	total: 43.7s	remaining: 45.6s
489:	learn: 0.0080106	total: 43.8s	remaining: 45.6s
490:	learn: 0.0079918	total: 43.9s	remaining: 45.5s
491:	learn: 0.0079677	total: 44s	remaining: 45.4s
492:	learn: 0.0079410	total: 44s	remaining: 45.3s
493:	learn: 0.0079286	total: 44.1s	remaining: 45.2s
494:	learn: 0.0079025	total: 44.2s	remaining: 45.1s
495:	learn: 0.0078787	total: 44.3s	remaining: 45s
496:	learn: 0.0078651	total: 44.4s	remaining: 44.9s
497:	learn: 0.0078603	

638:	learn: 0.0059592	total: 57.1s	remaining: 32.2s
639:	learn: 0.0059563	total: 57.1s	remaining: 32.1s
640:	learn: 0.0059446	total: 57.2s	remaining: 32s
641:	learn: 0.0059424	total: 57.3s	remaining: 31.9s
642:	learn: 0.0059309	total: 57.4s	remaining: 31.9s
643:	learn: 0.0059167	total: 57.5s	remaining: 31.8s
644:	learn: 0.0059061	total: 57.5s	remaining: 31.7s
645:	learn: 0.0058946	total: 57.7s	remaining: 31.6s
646:	learn: 0.0058871	total: 57.8s	remaining: 31.5s
647:	learn: 0.0058795	total: 57.9s	remaining: 31.4s
648:	learn: 0.0058774	total: 57.9s	remaining: 31.3s
649:	learn: 0.0058642	total: 58s	remaining: 31.2s
650:	learn: 0.0058520	total: 58.1s	remaining: 31.2s
651:	learn: 0.0058440	total: 58.2s	remaining: 31.1s
652:	learn: 0.0058414	total: 58.3s	remaining: 31s
653:	learn: 0.0058314	total: 58.4s	remaining: 30.9s
654:	learn: 0.0058179	total: 58.4s	remaining: 30.8s
655:	learn: 0.0058068	total: 58.5s	remaining: 30.7s
656:	learn: 0.0057925	total: 58.6s	remaining: 30.6s
657:	learn: 0.0057

...

450:	learn: 0.0091021	total: 40.4s	remaining: 49.2s
451:	learn: 0.0090742	total: 40.5s	remaining: 49.1s
452:	learn: 0.0090454	total: 40.6s	remaining: 49s
453:	learn: 0.0090288	total: 40.7s	remaining: 48.9s
454:	learn: 0.0090056	total: 40.7s	remaining: 48.8s
455:	learn: 0.0089739	total: 40.8s	remaining: 48.7s
456:	learn: 0.0089578	total: 41s	remaining: 48.7s
457:	learn: 0.0089380	total: 41.1s	remaining: 48.6s
458:	learn: 0.0089301	total: 41.2s	remaining: 48.5s
459:	learn: 0.0088962	total: 41.2s	remaining: 48.4s
460:	learn: 0.0088687	total: 41.3s	remaining: 48.3s
461:	learn: 0.0088432	total: 41.4s	remaining: 48.2s
462:	learn: 0.0088371	total: 41.5s	remaining: 48.1s
463:	learn: 0.0088129	total: 41.6s	remaining: 48s
464:	learn: 0.0087810	total: 41.6s	remaining: 47.9s
465:	learn: 0.0087607	total: 41.7s	remaining: 47.8s
466:	learn: 0.0087297	total: 41.8s	remaining: 47.7s
467:	learn: 0.0087041	total: 41.9s	remaining: 47.7s
468:	learn: 0.0086905	total: 42s	remaining: 47.6s
469:	learn: 0.008671

610:	learn: 0.0064090	total: 54.5s	remaining: 34.7s
611:	learn: 0.0063938	total: 54.6s	remaining: 34.6s
612:	learn: 0.0063790	total: 54.7s	remaining: 34.5s
613:	learn: 0.0063697	total: 54.7s	remaining: 34.4s
614:	learn: 0.0063668	total: 54.9s	remaining: 34.3s
615:	learn: 0.0063514	total: 55s	remaining: 34.3s
616:	learn: 0.0063368	total: 55.1s	remaining: 34.2s
617:	learn: 0.0063236	total: 55.1s	remaining: 34.1s
618:	learn: 0.0063142	total: 55.2s	remaining: 34s
619:	learn: 0.0062977	total: 55.3s	remaining: 33.9s
620:	learn: 0.0062828	total: 55.4s	remaining: 33.8s
621:	learn: 0.0062718	total: 55.5s	remaining: 33.7s
622:	learn: 0.0062635	total: 55.6s	remaining: 33.6s
623:	learn: 0.0062474	total: 55.6s	remaining: 33.5s
624:	learn: 0.0062348	total: 55.7s	remaining: 33.4s
625:	learn: 0.0062210	total: 55.8s	remaining: 33.4s
626:	learn: 0.0062177	total: 55.9s	remaining: 33.3s
627:	learn: 0.0062076	total: 56s	remaining: 33.2s
628:	learn: 0.0061934	total: 56.1s	remaining: 33.1s
629:	learn: 0.0061

769:	learn: 0.0049185	total: 1m 8s	remaining: 20.5s
770:	learn: 0.0049121	total: 1m 8s	remaining: 20.4s
771:	learn: 0.0049065	total: 1m 8s	remaining: 20.3s
772:	learn: 0.0048992	total: 1m 8s	remaining: 20.2s
773:	learn: 0.0048897	total: 1m 9s	remaining: 20.2s
774:	learn: 0.0048830	total: 1m 9s	remaining: 20.1s
775:	learn: 0.0048772	total: 1m 9s	remaining: 20s
776:	learn: 0.0048731	total: 1m 9s	remaining: 19.9s
777:	learn: 0.0048635	total: 1m 9s	remaining: 19.8s
778:	learn: 0.0048561	total: 1m 9s	remaining: 19.7s
779:	learn: 0.0048521	total: 1m 9s	remaining: 19.6s
780:	learn: 0.0048456	total: 1m 9s	remaining: 19.5s
781:	learn: 0.0048405	total: 1m 9s	remaining: 19.4s
782:	learn: 0.0048391	total: 1m 9s	remaining: 19.3s
783:	learn: 0.0048292	total: 1m 9s	remaining: 19.3s
784:	learn: 0.0048245	total: 1m 10s	remaining: 19.2s
785:	learn: 0.0048149	total: 1m 10s	remaining: 19.1s
786:	learn: 0.0048064	total: 1m 10s	remaining: 19s
787:	learn: 0.0048000	total: 1m 10s	remaining: 18.9s
788:	learn: 

925:	learn: 0.0040224	total: 1m 22s	remaining: 6.6s
926:	learn: 0.0040174	total: 1m 22s	remaining: 6.51s
927:	learn: 0.0040127	total: 1m 22s	remaining: 6.42s
928:	learn: 0.0040077	total: 1m 22s	remaining: 6.33s
929:	learn: 0.0040006	total: 1m 22s	remaining: 6.25s
930:	learn: 0.0039947	total: 1m 23s	remaining: 6.16s
931:	learn: 0.0039896	total: 1m 23s	remaining: 6.07s
932:	learn: 0.0039827	total: 1m 23s	remaining: 5.98s
933:	learn: 0.0039784	total: 1m 23s	remaining: 5.89s
934:	learn: 0.0039744	total: 1m 23s	remaining: 5.8s
935:	learn: 0.0039690	total: 1m 23s	remaining: 5.71s
936:	learn: 0.0039648	total: 1m 23s	remaining: 5.62s
937:	learn: 0.0039594	total: 1m 23s	remaining: 5.53s
938:	learn: 0.0039534	total: 1m 23s	remaining: 5.44s
939:	learn: 0.0039481	total: 1m 23s	remaining: 5.35s
940:	learn: 0.0039429	total: 1m 23s	remaining: 5.26s
941:	learn: 0.0039379	total: 1m 24s	remaining: 5.17s
942:	learn: 0.0039318	total: 1m 24s	remaining: 5.08s
943:	learn: 0.0039269	total: 1m 24s	remaining: 5

0.9970149253731343

In [15]:
cb_model.save_model('cb_model.cbm')

In [5]:
from lightgbm import Booster
lgbm_model = Booster(model_file='lgbm_model.txt')

In [5]:
cb_model = CatBoostClassifier()
cb_model.load_model('cb_model.cbm')

In [9]:
# 필요한 Mediapipe 솔루션을 초기화합니다.
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(max_num_hands=2, min_detection_confidence=0.5, min_tracking_confidence=0.5)
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()

# 비디오 캡처 시작
cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret, img = cap.read()
    if not ret:
        print("Ignoring empty camera frame.")
        continue

    # 이미지 처리
    img = cv2.flip(img, 1)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    result = hands.process(img)
    pose_result = pose.process(img)
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

    arm_angles = np.zeros(2)
    landmark_data = np.zeros(18)
    right_hand_angles = np.zeros(15)
    right_hand_coords = np.zeros(63)
    left_hand_angles = np.zeros(15)
    left_hand_coords = np.zeros(63)   
    
    if result.multi_hand_landmarks:
        for i, hand_landmarks in enumerate(result.multi_hand_landmarks):
            if i>= 2:
                break
            
            hand_type = result.multi_handedness[i].classification[0].label
            angles, joint_coords = calculate_angles(hand_landmarks, img.shape)

            if hand_type == 'Right':
                right_hand_angles = angles
                right_hand_coords = joint_coords
            elif hand_type == 'Left':
                left_hand_angles = angles
                left_hand_coords = joint_coords   
    
    if pose_result.pose_landmarks:
        arm_angles = calculate_pose_angles(pose_result.pose_landmarks, img.shape)
        landmark_data = []
        for idx in [11, 13, 15, 12, 14, 16]:
            landmark = pose_result.pose_landmarks.landmark[idx]
            landmark_data.extend([landmark.x, landmark.y, landmark.z])
        if len(landmark_data) == 0:
            landmark_data = np.zeros(18)
            
    data = np.concatenate((right_hand_angles, right_hand_coords, left_hand_angles, left_hand_coords, arm_angles, landmark_data))
        
    data = data.reshape(1,-1)
    predicted_label = cb_model.predict(data)
    
    print(predicted_label[0][0])
    # 예측된 라벨을 화면에 표시
    cv2.putText(img, text=str(predicted_label[0]), org=(50, 50),
                fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=1, color=(0, 0,0), thickness=2)
    
    # 손 랜드마크 그리기
    if result.multi_hand_landmarks:
        for res in result.multi_hand_landmarks:
            mp_drawing.draw_landmarks(img, res, mp_hands.HAND_CONNECTIONS)
    
    # 화면에 이미지 표시
    cv2.imshow('img', img)
    if cv2.waitKey(30) == 49:  # 1번 키를 누르면 종료
        break
        
cap.release()
cv2.destroyAllWindows()

wallet
toilet
toilet
toilet
toilet
toilet
toilet
toilet
toilet
toilet
toilet
toilet
toilet
toilet
toilet
toilet
toilet
toilet
toilet
toilet
toilet
toilet
sick
report
sick
toilet
report
report
report
report
sick
report
report
down
toilet
toilet
report
down
toilet
down
report
report
report
sick
toilet
toilet
report
toilet
report
down
toilet
report
down
down
report
report
report
report
report
report
report
report
report
report
report
report
report
report
child
report
report
report
child
report
child
child
report
report
report
report
report
report
child
report
report
sick
report
toilet
report
report
report
sick
sick
report
report
where
where
toilet
report
report
toilet
toilet
toilet
toilet
toilet
sick
report
report
toilet
toilet
toilet
toilet
toilet
toilet
toilet
toilet
toilet
toilet
toilet
toilet
toilet
report
toilet
report
toilet
report
report
sick
report
report
down
report
child
sick
report
sick
report
report
report
report
report
toilet
report
report
sick
toilet
toilet
toilet
toilet
toi